In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import os

def train_and_save_model(model, data, username, epochs=10, batch_size=32):
    # Compile the model with a binary cross-entropy loss function and accuracy metric
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model on the keystroke data
    model.fit(data, data, epochs=epochs, batch_size=batch_size)
    
    # Save the trained model to a file and return the file path
    model_path = f'models/{username}_model.h5'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model.save(model_path)
    
    return model_path

def create_cnn_model(input_shape):
    # Initialize the CNN model
    model = Sequential()
    
    # Add convolutional and pooling layers
    model.add(Conv2D(9, kernel_size=(9), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the output and add dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    
    return model


def process_data(data = []):
    rf = np.array(data)
    keystrokes = []
    press_times = []
    release_times = []
    
    for k in rf:
        keystrokes.append(k[0])
        if k[2] == 'keydown':
            press_times.append(k[1])
        elif k[2] == 'keyup':
            release_times.append(k[1])
    
    print('keystrokes: ',len(keystrokes))
    print('\n')
    print('press times: ',len(press_times))
    print('\n')
    print('released times: ',len(release_times))
    flight_times = np.array(release_times) - np.array(press_times)
    
    print('released times: ',len(flight_times),flight_times)
    
    delayed_times = np.array(press_times[1:]) - np.array(release_times[:-1])
    print('\n')
    print('delayed times: ',len(delayed_times),delayed_times)
    
    new_data = np.concatenate([flight_times,delayed_times])
    print('\n')
    print('fight/delayed times: ',len(new_data),new_data.shape,new_data)
    reshaped_data = new_data.reshape(-1,1)
    print('\n')
    print('reshaped fight/delayed times: ',len(reshaped_data),reshaped_data.shape,reshaped_data)
    
    model = create_cnn_model(input_shape=reshaped_data.shape)
    model_path = train_and_save_model(model,reshaped_data,'GOD IS GOOD')
    
    print('\n')
    print(f'model path: {model_path}')
    
    

    

def process_items():
    data = pd.read_csv("../data/raw/GOD IS GOOD.csv")
    process_data(data)
    # print(data)
    
if __name__ == "__main__":
    process_items()
    


keystrokes:  248


press times:  124


released times:  124
released times:  124 [139 157 297 251 187 172 235 203 109 188 188 187 155 157 343 296 203 156
 234 312 266 218 281 187 172 235 250 187 219 250 187 156 188 219 250 188
 172 203 219 344 203 266 313 250 391 234 188 203  94 234 172 141 218 406
 297 343 359 391 250 250 157 172 109 140 250 188 188 282 234 188 234 172
 312 141 250 234 250 218 250 297 297 282 422 360 125 187 157 250 390 203
 125 186 187 203 203 234 375 281 235 250 172 234 234 313 328 219  94 235
 234 125 156 219 281 281 312 203 235 234 219 235 188 281 250 172]


delayed times:  123 [ 266   30 -125 -172  -94  110 -173  -93   94 -173  -62  110   79  -48
 -155 -172 -156   32  -78 -250 -109 -156  -62 -172   47 -157 -187 -172
 -219   32  -78  -78 -188  -78  -63 -172 -172  -94 -203 -125 -188 -188
 -203  -47 -172  -94 -188   63   31 -172  -31   47 -140  -31 -297 -343
 -344    0  -16 -125  -63  -47  -15  -93 -188 -157  -63  -94 -172 -172
  -78 -140    0  -16 -156  -62  -78 -2

d:\development\laragon\www\v2keystrokes_auth_python\env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 247, 1)